<a href="https://colab.research.google.com/github/alkarps/GB.Methods-for-collecting-and-processing-data-from-the-Internet/blob/hw02/hw02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install requests
#!pip install lxml

In [2]:
from pprint import pprint
from lxml import html
import requests

In [3]:
class New:
    def __init__(self, source, title, link, date):
        self.source = source
        self.title = title
        self.link = link
        self.date = date
    def __str__(self):
        return f"source={self.source}; title={self.title}; link={self.link}; date={self.date}"

Написать приложение или функцию, которые собирают основные новости с сайтов на выбор lenta.ru, yandex-новости. Для парсинга использовать XPath.

Структура данных должны содержать:
 - *название источника;
 - наименование новости;
 - ссылку на новость;
 - дата публикации.


 Минимум один сайт, максимум - все два.

In [4]:
lenta_url = "https://lenta.ru"
lenta_uri_news = "//a[contains(@class, '_topnews') or contains(@class, '_longgrid')  or contains(@class, 'card-feature') ]"

In [28]:
res = requests.get(lenta_url)

In [39]:
dom = html.fromstring(res.text)

In [40]:
news = dom.xpath(lenta_uri_news)

In [77]:
def parseNewForLenta(new):
  link =  new.attrib["href"]
  if link.startswith("/"):
    link = lenta_url + link
  source = link.split("/")[2]
  date = ""
  title = ""
  if "card-mini" in new.classes:
    card_mini = new.getiterator()
    for ch in card_mini:
      if "card-mini__date" in ch.classes:
        date = ch.text
      elif "card-mini__title" in ch.classes:
        title = ch.text
  elif "card-feature" in new.classes:
    card_feature = new.getiterator()
    sub_title = ""
    for ch in card_feature:
      if "card-feature__date" in ch.classes:
        date = ch.text
      elif "card-feature__title" in ch.classes:
        main_title = ch.text
      elif "card-feature__rightcol" in ch.classes:
        sub_title = ch.text
    divider = ":"
    if main_title.endswith((".","?","!")):
      divider = " "
    title = f"{main_title}{divider}{sub_title}"
  else:
    card_big = new.getiterator()
    sub_title = ""
    for ch in card_big:
      if "card-big__date" in ch.classes:
        date = ch.text
      elif "card-big__title" in ch.classes:
        main_title = ch.text
      elif "card-big__rightcol" in ch.classes:
        sub_title = ch.text
    divider = ":"
    if main_title.endswith((".","?","!")):
      divider = " "
    title = f"{main_title}{divider}{sub_title}"
  return New(source, title, link, date)

In [78]:
result = list()
for new in news:
  _new = parseNewForLenta(new)
  print (_new)
  result.append(_new)
print(len(result))

source=lenta.ru; title=Путин пообещал «пощелкать» американские «Патриоты» на Украине:; link=https://lenta.ru/news/2022/12/22/patttt/; date=19:17
source=lenta.ru; title=Путин спрогнозировал уровень инфляции по итогам года; link=https://lenta.ru/news/2022/12/22/prognoss/; date=19:25
source=lenta.ru; title=Врач назвал диету и спорт неэффективными способами борьбы с ожирением; link=https://lenta.ru/news/2022/12/22/obesity/; date=19:21
source=lenta.ru; title=Перечислены лучшие гаджеты 2022 года; link=https://lenta.ru/news/2022/12/22/best_gadgets/; date=19:14
source=lenta.ru; title=Путин рассказал о ежедневных встречах с российскими военными; link=https://lenta.ru/news/2022/12/22/putvoen/; date=19:12
source=moslenta.ru; title=Известная актриса призналась в необычной для своей профессии фобии; link=https://moslenta.ru/news/lyudi/aktrisa-22-12-2022.htm; date=19:09
source=lenta.ru; title=Путин назвал странным отказ Киева от переговоров с Москвой; link=https://lenta.ru/news/2022/12/22/otkaz/; da